In [1]:
#LOAD LIBRARIES
import pandas as pd                     #Library pengolahan data frame
from google.colab import files          #Library download data ke local drives
from google.colab import auth           #Library akses google colab ke akun gmail supaya bisa connect ke google sheets
auth.authenticate_user()                #Otorisasi akun gmail
from google.auth import default         #Library otorisasi default
import gspread                          #Library impor dan ekspor data dari/ke google sheet
creds, _ = default()                    #Credentials library gspread
gc = gspread.authorize(creds)           #Otorisasi library gspread

#Libaries Khusus naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.preprocessing import OneHotEncoder

#Library Visualisasi Data
import seaborn as sn
import matplotlib.pyplot as plt

In [2]:
#IMPORT DATA
gc_posbindu = gc.open_by_url('https://docs.google.com/spreadsheets/d/1GMO75VfR3CPDy8swwGrmREc_LtfiGd05MSS3yAgzYog/edit#gid=53291931') #Link ke google sheets kunjungan portal

#Penyusunan dataframe pasien semua jenis kelamin
gc_analisa = gc_posbindu.worksheet('to Python')
pasien_lists = gc_analisa.get_all_values()
pasien = pd.DataFrame(pasien_lists[1:])
pasien.columns = pasien_lists[0]

#Penyusunan dataframe pasien laki-laki
pasien_l = pasien.loc[pasien['JK'] == 'Laki-Laki']

#Penyusunan dataframe pasien perempuan
pasien_p = pasien.loc[pasien['JK'] == 'Perempuan']
pasien.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   No Urut                        87 non-null     object
 1   Nama                           87 non-null     object
 2   Umur                           87 non-null     object
 3   JK                             87 non-null     object
 4   PTM Keluarga                   87 non-null     object
 5   PTM Diri                       87 non-null     object
 6   Pernah Aktif Merokok           87 non-null     object
 7   Pernah Kurang Aktivitas Fisik  87 non-null     object
 8   Pernah Kurang sayur/buah       87 non-null     object
 9   Systolic avg                   87 non-null     object
 10  Diastolic avg                  87 non-null     object
 11  Klasifikasi                    87 non-null     object
 12  Klasifikasi 2                  87 non-null     object
dtypes: obje

In [3]:
#plt.bar(pasien_l['Klasifikasi'],pasien_l['Diastolic avg'])
#plt.show

pd.crosstab(pasien_l['Klasifikasi 2'],pasien_l['Pernah Kurang sayur/buah'], normalize='index')

Pernah Kurang sayur/buah,Sering,Tidak
Klasifikasi 2,,
Gizi Lebih/Obesitas,0.285714,0.714286
Normal,0.666667,0.333333


In [4]:
pasien = pd.get_dummies(pasien, prefix='JK', columns=['JK'])
pasien = pd.get_dummies(pasien, prefix='PTM Keluarga', columns=['PTM Keluarga'])
pasien = pd.get_dummies(pasien, prefix='PTM Diri', columns=['PTM Diri'])
pasien = pd.get_dummies(pasien, prefix='Rokok', columns=['Pernah Aktif Merokok'])
pasien = pd.get_dummies(pasien, prefix='Kurang Akt Fisik', columns=['Pernah Kurang Aktivitas Fisik'])
pasien = pd.get_dummies(pasien, prefix='Kurang Sayur', columns=['Pernah Kurang sayur/buah'])
pasien.info()

#Penyusunan dataframe pasien laki-laki
pasien_l = pasien.loc[pasien['JK_Laki-Laki'] == 1]

#Penyusunan dataframe pasien perempuan
pasien_p = pasien.loc[pasien['JK_Perempuan'] == 1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   No Urut                     87 non-null     object
 1   Nama                        87 non-null     object
 2   Umur                        87 non-null     object
 3   Systolic avg                87 non-null     object
 4   Diastolic avg               87 non-null     object
 5   Klasifikasi                 87 non-null     object
 6   Klasifikasi 2               87 non-null     object
 7   JK_Laki-Laki                87 non-null     uint8 
 8   JK_Perempuan                87 non-null     uint8 
 9   PTM Keluarga_-              87 non-null     uint8 
 10  PTM Keluarga_Diabet         87 non-null     uint8 
 11  PTM Keluarga_Hipertensi     87 non-null     uint8 
 12  PTM Keluarga_Kanker Servik  87 non-null     uint8 
 13  PTM Keluarga_Leukimia       87 non-null     uint8 
 

In [31]:
#Pemisahan dataframe pasien semua jenis kelamin
pasien_X = pasien[['Umur','JK_Laki-Laki','PTM Keluarga_Diabet',
                       'PTM Keluarga_Hipertensi','PTM Keluarga_Kanker Servik','PTM Keluarga_Leukimia',
                       'PTM Diri_Diabet','PTM Diri_Hipertensi','Rokok_Pernah','Rokok_Sering',
                       'Kurang Akt Fisik_Pernah','Kurang Akt Fisik_Sering','Kurang Sayur_Pernah','Kurang Sayur_Sering']]
pasien_y = pasien['Klasifikasi 2']
pasien_X_train, pasien_X_test, pasien_y_train, pasien_y_test = train_test_split(pasien_X, pasien_y, test_size=0.20, random_state=10)

#Pemisahan dataframe pasien laki-laki
pasien_l_X = pasien_l[['Umur','Systolic avg','Diastolic avg','PTM Keluarga_Diabet',
                       'PTM Keluarga_Hipertensi','PTM Keluarga_Kanker Servik','PTM Keluarga_Leukimia',
                       'PTM Diri_Diabet','PTM Diri_Hipertensi','Rokok_Pernah','Rokok_Sering',
                       'Kurang Akt Fisik_Pernah','Kurang Akt Fisik_Sering','Kurang Sayur_Pernah','Kurang Sayur_Sering']]
pasien_l_y = pasien_l['Klasifikasi 2']
pasien_l_X_train, pasien_l_X_test, pasien_l_y_train, pasien_l_y_test = train_test_split(pasien_l_X, pasien_l_y, test_size=0.2, random_state=10)

#Pemisahan dataframe pasien perempuan
pasien_p_X = pasien_p[['Umur','Systolic avg','Diastolic avg','PTM Keluarga_Diabet',
                       'PTM Keluarga_Hipertensi','PTM Keluarga_Kanker Servik','PTM Keluarga_Leukimia',
                       'PTM Diri_Diabet','PTM Diri_Hipertensi','Rokok_Pernah','Rokok_Sering',
                       'Kurang Akt Fisik_Pernah','Kurang Akt Fisik_Sering','Kurang Sayur_Pernah','Kurang Sayur_Sering']]
pasien_p_y = pasien_p['Klasifikasi 2']
pasien_p_X_train, pasien_p_X_test, pasien_p_y_train, pasien_p_y_test = train_test_split(pasien_p_X, pasien_p_y, test_size=0.2, random_state=10)


In [53]:
pasien['Rokok_Pernah'] = pasien['Rokok_Pernah'] + pasien['Rokok_Sering']
pasien['Kurang Akt Fisik_Pernah'] = pasien['Kurang Akt Fisik_Pernah'] + pasien['Kurang Akt Fisik_Sering']
pasien['Kurang Sayur_Pernah'] = pasien['Kurang Sayur_Pernah'] + pasien['Kurang Sayur_Sering']

In [57]:
#Pemisahan dataframe pasien semua jenis kelamin
pasien_X = pasien[['Umur','JK_Laki-Laki','PTM Keluarga_Diabet',
                       'PTM Keluarga_Hipertensi','PTM Keluarga_Kanker Servik','PTM Keluarga_Leukimia',
                       'PTM Diri_Diabet','PTM Diri_Hipertensi','Rokok_Pernah','Rokok_Sering',
                       'Kurang Akt Fisik_Pernah','Kurang Akt Fisik_Sering','Kurang Sayur_Pernah','Kurang Sayur_Sering']]
pasien_y = pasien['Klasifikasi 2']
pasien_X_train, pasien_X_test, pasien_y_train, pasien_y_test = train_test_split(pasien_X, pasien_y, test_size=0.20, random_state=10)

gnb = GaussianNB()
pasien_y_pred = gnb.fit(pasien_X_train, pasien_y_train).predict(pasien_X_test)
print(pasien_y_pred)
print("Number of mislabeled points out of a total %d points : %d" % (pasien_X_test.shape[0], (pasien_y_test != pasien_y_pred).sum()))
print("Accuracy : %f" % ((pasien_y_test == pasien_y_pred).sum()/pasien_X_test.shape[0]))
print(classification_report(pasien_y_test, pasien_y_pred))
confusion_matrix(pasien_y_test, pasien_y_pred)

['Gizi Lebih/Obesitas' 'Gizi Lebih/Obesitas' 'Gizi Lebih/Obesitas'
 'Gizi Lebih/Obesitas' 'Gizi Lebih/Obesitas' 'Gizi Lebih/Obesitas'
 'Gizi Lebih/Obesitas' 'Gizi Lebih/Obesitas' 'Gizi Lebih/Obesitas'
 'Gizi Lebih/Obesitas' 'Gizi Lebih/Obesitas' 'Gizi Lebih/Obesitas'
 'Gizi Lebih/Obesitas' 'Normal' 'Gizi Lebih/Obesitas'
 'Gizi Lebih/Obesitas' 'Gizi Lebih/Obesitas' 'Gizi Lebih/Obesitas']
Number of mislabeled points out of a total 18 points : 3
Accuracy : 0.833333
                     precision    recall  f1-score   support

Gizi Lebih/Obesitas       0.82      1.00      0.90        14
             Normal       1.00      0.25      0.40         4

           accuracy                           0.83        18
          macro avg       0.91      0.62      0.65        18
       weighted avg       0.86      0.83      0.79        18



array([[14,  0],
       [ 3,  1]])

In [ ]:
pasien_l_y_pred = gnb.fit(pasien_l_X_train, pasien_l_y_train).predict(pasien_l_X_test)
print(pasien_l_y_pred)
print("Number of mislabeled points out of a total %d points : %d" % (pasien_l_X_test.shape[0], (pasien_l_y_test != pasien_l_y_pred).sum()))
print("Accuracy : %f" % ((pasien_l_y_test == pasien_l_y_pred).sum()/pasien_l_X_test.shape[0]))
print(classification_report(pasien_l_y_test, pasien_l_y_pred))
confusion_matrix(pasien_l_y_test, pasien_l_y_pred)

['Normal' 'Normal' 'Normal' 'Normal' 'Gizi Lebih/Obesitas' 'Normal']
Number of mislabeled points out of a total 6 points : 3
Accuracy : 0.500000
                     precision    recall  f1-score   support

Gizi Lebih/Obesitas       0.00      0.00      0.00         2
             Normal       0.60      0.75      0.67         4

           accuracy                           0.50         6
          macro avg       0.30      0.38      0.33         6
       weighted avg       0.40      0.50      0.44         6



array([[0, 2],
       [1, 3]])

In [ ]:
pasien_p_y_pred = gnb.fit(pasien_p_X_train, pasien_p_y_train).predict(pasien_p_X_test)
print(pasien_p_y_pred)
print("Number of mislabeled points out of a total %d points : %d" % (pasien_p_X_test.shape[0], (pasien_p_y_test != pasien_p_y_pred).sum()))
print("Accuracy : %f" % ((pasien_p_y_test == pasien_p_y_pred).sum()/pasien_p_X_test.shape[0]))
print(classification_report(pasien_p_y_test, pasien_p_y_pred))
confusion_matrix(pasien_p_y_test, pasien_p_y_pred)

['Normal' 'Gizi Lebih/Obesitas' 'Normal' 'Normal' 'Normal' 'Normal'
 'Normal' 'Normal' 'Normal' 'Gizi Lebih/Obesitas' 'Normal'
 'Gizi Lebih/Obesitas' 'Gizi Lebih/Obesitas']
Number of mislabeled points out of a total 13 points : 7
Accuracy : 0.461538
                     precision    recall  f1-score   support

Gizi Lebih/Obesitas       1.00      0.36      0.53        11
             Normal       0.22      1.00      0.36         2

           accuracy                           0.46        13
          macro avg       0.61      0.68      0.45        13
       weighted avg       0.88      0.46      0.51        13



array([[4, 7],
       [0, 2]])